In [2]:
# Install required packages
!pip install -q einops==0.7.0 langchain==0.1.9 pypdf==4.0.2 sentence-transformers==2.4.0
!pip install mysql-connector-python

import os
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.vectorstores import FAISS
from langchain.schema import Document
import mysql.connector

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# OpenAI API key
OPENAI_API_KEY = ""  # Replace with your OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Model parameters
MAX_TOKENS = 1024
TOP_P = 0.95
TEMPERATURE = 0.01
PRESENCE_PENALTY = 1.03

# Embedding model configuration
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



<All keys matched successfully>


In [4]:
# Connect to MariaDB and load documents
db_connection = mysql.connector.connect(
    host="mariadb.banco-galicia.svc.cluster.local",
    user="",
    password="",
    database=""
)
cursor = db_connection.cursor()

# Fetch documents from the database
cursor.execute("SELECT source, content FROM pdf_documents")
rows = cursor.fetchall()

In [8]:
!pip install faiss-cpu

# Create Document objects
documents = []
for row in rows:
    source, content = row
    metadata = {'source': source}
    doc = Document(page_content=content, metadata=metadata)
    documents.append(doc)

cursor.close()
db_connection.close()

# Create a vector store using FAISS
vectorstore = FAISS.from_documents(documents, embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 200.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [9]:
# Define the system prompt
system_template = """You are a helpful, respectful, and honest assistant named HatBot answering questions.
You will be given a question and a context to provide you with information. You must answer the question based as much as possible on this context.
Always answer as helpfully as possible while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense or is not factually coherent, explain why instead of providing incorrect information. If you don't know the answer to a question, please don't share false information."""

# Define the user prompt
human_template = """Context:
{context}

Question: {question}"""

# Create the chat prompt template
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

In [10]:
# Initialize the OpenAI ChatGPT model
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4o",
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    temperature=TEMPERATURE,
    presence_penalty=PRESENCE_PENALTY,
    streaming=True,
    verbose=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}
    ),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [11]:
# Question
question = "Explícame sobre el ARTÍCULO SEGUNDO del estatuto Loma Negra?"
result = qa_chain({"query": question})

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


El ARTÍCULO SEGUNDO del estatuto de Loma Negra Compañía Industrial Argentina Sociedad Anónima establece lo siguiente:

1. **Domicilio Legal**: La sociedad tiene su domicilio legal en la Ciudad Autónoma de Buenos Aires.
2. **Filiales y Sucursales**: A pesar de tener su domicilio principal en Buenos Aires, el Directorio de la sociedad tiene la facultad de establecer filiales, agencias, sucursales y domicilios especiales en cualquier punto de la República Argentina o en el extranjero.
3. **Capital Determinado**: El Directorio puede decidir si estas filiales, agencias o sucursales tendrán un capital determinado o no.
4. **Leyes Extranjeras**: El domicilio fijado en Buenos Aires se establece sin perjuicio de las leyes de otros países que puedan afectar a las filiales, sucursales y agencias establecidas fuera de Argentina.

En resumen, este artículo define el domicilio principal de la sociedad y otorga al Directorio la flexibilidad para expandir sus operaciones tanto dentro como fuera del pa

In [ ]:
# Function to remove duplicate documents
def remove_duplicates(input_list):
    unique_list = []
    sources = set()
    for item in input_list:
        if item.metadata['source'] not in sources:
            unique_list.append(item)
            sources.add(item.metadata['source'])
    return unique_list

# Process and print results
results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)
